# BEAUTIFUL SOUP [API](https://beautiful-soup-4.readthedocs.io/en/latest/)

# <b>1. Objects</b>

Beautiful Soup transforms a complex HTML document into a complex tree of Python objects. But you’ll only ever have to deal with about four kinds of objects: 
- `Tag`, 
- `NavigableString`, 
- `BeautifulSoup`, and 
- `Comment`.

# 1.1 Tag

A `Tag` object corresponds to an XML or HTML tag in the original document:

In [9]:
from bs4 import BeautifulSoup

soup = BeautifulSoup('<b class="boldest">Extremely bold</b>')
tag = soup.b
print(type(tag))

<class 'bs4.element.Tag'>


Tags have a lot of attributes and methods, and I’ll cover most of them in [Navigating the tree](#) and [Searching the tree](#). For now, the most important features of a tag are its 
- **name** and 
- **attributes**.

## `tag.name`

In [10]:
tag.name

'b'

If you change a tag’s name, the change will be reflected in any HTML markup generated by Beautiful Soup:

In [11]:
tag.name = "blockquote"
tag

<blockquote class="boldest">Extremely bold</blockquote>

## Attributes

A tag may have any number of attributes. The tag `<b class="boldest">` has an attribute “`class`” whose value is “`boldest`”. You can access a tag’s attributes by treating the tag like a dictionary:

In [19]:
tag = soup.b
print(tag)
tag['class']

<b class="boldest">Extremely bold</b>


['boldest']

In [20]:
tag.attrs

{'class': ['boldest']}

You can add, remove, and modify a tag’s attributes. Again, this is done by treating the tag as a dictionary:

In [23]:
tag['id'] = 'verybold'
tag['another-attribute'] = 1
tag

<b another-attribute="1" class="boldest" id="verybold">Extremely bold</b>

In [24]:
del tag['id']
del tag["another-attribute"]
tag

<b class="boldest">Extremely bold</b>

### Multi-valued attributes

HTML 4 defines a few attributes that can have multiple values. HTML 5 removes a couple of them, but defines a few more. The most common multi-valued attribute is `class` (that is, a tag can have more than one CSS class). Others include `rel`, `rev`, `accept-charset`, `headers`, and `accesskey`. Beautiful Soup presents the value(s) of a multi-valued attribute as a list.

Defaults implement the rules described in the HTML specification:

In [40]:
from bs4.builder import builder_registry
builder_registry.lookup('html').DEFAULT_CDATA_LIST_ATTRIBUTES

{'*': ['class', 'accesskey', 'dropzone'],
 'a': ['rel', 'rev'],
 'link': ['rel', 'rev'],
 'td': ['headers'],
 'th': ['headers'],
 'form': ['accept-charset'],
 'object': ['archive'],
 'area': ['rel'],
 'icon': ['sizes'],
 'iframe': ['sandbox'],
 'output': ['for']}

In [26]:
css_soup = BeautifulSoup('<p class="body strikeout"></p>')
css_soup.p.attrs

{'class': ['body', 'strikeout']}

In [27]:
css_soup.p["class"]

['body', 'strikeout']

If an attribute looks like it has more than one value, but it’s not a multi-valued attribute as defined by any version of the HTML standard, Beautiful Soup will leave the attribute alone:

In [28]:
id_soup = BeautifulSoup('<p id="my id"></p>')
id_soup.p['id']

'my id'

When you turn a tag back into a string, multiple attribute values are consolidated:

In [34]:
rel_soup = BeautifulSoup('<p>Back to the <a rel="index">homepage</a></p>')
print(rel_soup.a)

rel_soup.a["rel"] = ['index', 'content']
print(rel_soup.a)

<a rel="index">homepage</a>
<a rel="index content">homepage</a>


You can disable this by passing `multi_valued_attributes=None` as a keyword argument into the BeautifulSoup constructor:

In [37]:
no_list_soup = BeautifulSoup(
    '<p class="body strikeout"></p>', 'html', 
        multi_valued_attributes=None)

no_list_soup.p["class"]

'body strikeout'

You can use `get_attribute_list` to get a value that’s always a list, whether or not it’s a multi-valued atribute:

In [38]:
id_soup.p.get_attribute_list("id")

['my id']

If you parse a document as XML, there are no multi-valued attributes, because it does not use the HTML specification:

In [39]:
xml_soup = BeautifulSoup('<p class="body strikeout"></p>', 'xml')
xml_soup.p["class"]

'body strikeout'

In [43]:
xml_soup = BeautifulSoup('<p class="body strikeout"></p>', 'lxml')
xml_soup.p["class"]

['body', 'strikeout']

Again, you can configure this using the `multi_valued_attributes` argument:

In [42]:
class_is_multi= { '*' : 'class'}
xml_soup = BeautifulSoup('<p class="body strikeout"></p>', 
                         'xml',
                         multi_valued_attributes=class_is_multi)
xml_soup.p["class"]

['body', 'strikeout']

# <b>Filters</b>

## Searching by CSS class

It’s very useful to search for a tag that has a certain CSS class. But the name of the CSS attribute, “class”, is a reserved word in Python. Using `class` as a keyword argument will give you a syntax error. As of Beautiful Soup 4.1.2, you can search by CSS class using the keyword argument `class_`:

```python
soup.find_all("a", class_="sister")
# [<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
#  <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
#  <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]
```

As with any keyword argument, you can pass `class_` 
- a string, 
- a regular expression, 
- a function, or 
- `True`:

```python
soup.find_all(class_=re.compile("itl"))
# [<p class="title"><b>The Dormouse's story</b></p>]

def has_six_characters(css_class):
    return css_class is not None and len(css_class) == 6

soup.find_all(class_=has_six_characters)
# [<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
#  <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
#  <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]
```
***
_ChatGPT:_  
In the code you provided, you are using the `find_all` method of BeautifulSoup to find all HTML elements that match a certain condition based on their **class attribute**. The `class_` parameter is used to filter elements based on their class attribute. In the first example, you are using a regular expression to find elements whose class attribute contains the substring "`itl`".

In the second example, you define a custom function `has_six_characters` which takes a `css_class` argument and returns `True` if the length of the class attribute is `6` and it is not `None`. Then, you pass this function as the `class_` parameter to `find_all`, effectively using it as a filter function.

Now, let's address your question about why you don't pass any arguments into `class_=has_six_characters`. When you pass a function as the value of `class_ parameter` in BeautifulSoup, it internally calls that function for each element's class attribute in the HTML document. It automatically passes the class attribute of each element as an argument to the function. In your case, `has_six_characters` is a function that takes a single argument (`css_class`), but you don't need to explicitly pass that argument when using it with `find_all`.

The `find_all` method takes care of calling your function with the appropriate arguments (in this case, the class attribute of each HTML element). The result is a filtered list of elements whose class attribute satisfies the condition specified in your custom function.

If you define a function with two arguments and you want to use it as a filter function with BeautifulSoup's `find_all` method, the function should still only take one argument, which will be automatically supplied by BeautifulSoup.

For example, consider a function with two arguments:

```python
def has_specific_class(css_class, target_class):
    return css_class is not None and target_class in css_class
```

If you want to use this function with `find_all`, you should only include the function name (`has_specific_class`) in the `class_` parameter, and BeautifulSoup will automatically pass the class attribute as the first argument. The second argument (`target_class` in this example) needs to be fixed when calling find_all:

```python
target_class = "example_class"
soup.find_all(class_=lambda css_class: has_specific_class(css_class, target_class))
```

In this example, `lambda css_class: has_specific_class(css_class, target_class)` is an anonymous function (lambda function) that takes one argument (the class attribute) and passes it to the `has_specific_class` function along with the fixed `target_class`. This way, `find_all` can still use the function as a filter for elements based on their class attributes.
***

Remember that a single tag can have multiple values for its “class” attribute. When you search for a tag that matches a certain CSS class, you’re matching against any of its CSS classes:

In [3]:
from bs4 import BeautifulSoup

css_soup = BeautifulSoup('<p class="body strikeout"></p>')

In [4]:
css_soup.find_all("p", class_="strikeout")

[<p class="body strikeout"></p>]

In [5]:
css_soup.find_all("p", class_="body")

[<p class="body strikeout"></p>]

You can also search for the exact string value of the class attribute:

In [6]:
css_soup.find_all("p", class_="body strikeout")

[<p class="body strikeout"></p>]

But searching for variants of the string value won’t work:

In [7]:
css_soup.find_all("p", class_="strikeout body")

[]

If you want to search for tags that match two or more CSS classes, you should use a CSS selector:

In [8]:
css_soup.select("p.strikeout.body")

[<p class="body strikeout"></p>]

In older versions of Beautiful Soup, which don’t have the `class_` shortcut, you can use the `attrs` trick mentioned above. Create a dictionary whose value for “class” is the string (or regular expression, or whatever) you want to search for:

```python
soup.find_all("a", attrs={"class": "sister"})
# [<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
#  <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
#  <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]
```